In [1]:
import warnings

warnings.filterwarnings("ignore")

import os
import re
import sys
from pathlib import Path

import numpy as np
import pandas as pd
from rich import print
from tqdm import tqdm

In [2]:
def read_file_to_str_li(fp, print_exp=True):
    with open(fp, "r") as f:
        lines = f.read().split("\n")
    if print_exp:
        print(f"Read from {fp}:")
        print(f"First line: {lines[0]} | Last line: {lines[-1]}")
        print("-" * 6)

    return lines


# define the function blocks
def convert_to_int(input_str):
    if input_str == "" or input_str == " ":
        return None
    return int(input_str)


def convert_to_str(input_str):
    return str(input_str)


# map the inputs to the function blocks
converts = {
    "i": convert_to_int,
    "s": convert_to_str,
}


def convert_str_li_to_other_li(
    str_li, pattern="i", per_letter=False, sep=" ", start_row=0, end_row=None
):
    """ Convert a list of string to a list of other types
    
    pattern: a list of types for one item. 
        'i' for int, 's' for string
        'si' means: convert the 1st item to string, the rest to integer
        If separated items are more than pattern items,
        use the last one from the parttern.
    if per_letter=True, ignore sep and separate item per letter
    """
    target_str_li = str_li[start_row:end_row]
    # find max item num
    max_item_num = 1
    if per_letter:
        max_item_num = max([len(s) for s in target_str_li])
    else:
        max_item_num = max([len(s.split(sep)) for s in target_str_li])

    # extend the pattern to the max itme num
    pattern = (
        pattern + f"{pattern[-1]}" * (max_item_num - len(pattern))
        if max_item_num > len(pattern)
        else pattern
    )

    # convert
    if per_letter:
        return [
            [converts[pattern[idx]](item) for idx, item in enumerate(s)]
            for s in target_str_li
        ]
    else:
        if sep == " ":
            return [
                [converts[pattern[idx]](item) for idx, item in enumerate(s.split())]
                for s in target_str_li
            ]
        else:
            return [
                [converts[pattern[idx]](item) for idx, item in enumerate(s.split(sep))]
                for s in target_str_li
            ]

In [3]:
fp = "input.txt"
lines = read_file_to_str_li(fp)

print("Convert to:")

# head = convert_str_li_to_other_li(
#     lines, pattern="i", per_letter=False, sep=",", start_row=0, end_row=1
# )

# print(f"Head:\n{head}")

data = convert_str_li_to_other_li(
    lines, pattern="s", per_letter=False, sep=" -> ", start_row=None, end_row=None
)
# data = convert_str_li_to_other_li(
#     lines, pattern="i", per_letter=False, sep=" ", start_row=2, end_row=None
# )
# data = convert_str_li_to_other_li(
#     lines, pattern="si", per_letter=False, sep=" ", start_row=0, end_row=None
# )
# data = convert_str_li_to_other_li(
#     lines, pattern="i", per_letter=True, sep=" ", start_row=0, end_row=None
# )

print(f"First line: {data[0]}")
print(f"Last line: {data[-1]}")
print("-" * 6)

Read from input.txt:

First line: 529,822 -> 529,562 | Last line: 906,28 -> 906,957

------

Convert to:

First line: ['529,822', '529,562']

Last line: ['906,28', '906,957']

------

In [4]:
from math import gcd


class Line:
    def __init__(self, input_coord):
        self.x1 = int(input_coord[0].split(",")[0])
        self.y1 = int(input_coord[0].split(",")[1])
        self.x2 = int(input_coord[1].split(",")[0])
        self.y2 = int(input_coord[1].split(",")[1])

    def check_hor_or_ver(self):
        if self.x1 == self.x2 or self.y1 == self.y2:
            return True
        else:
            return False

    def max_coord_x(self):
        return max(self.x1, self.x2)

    def max_coord_y(self):
        return max(self.y1, self.y2)

    def covered_points(self):
        rel_x = self.x2 - self.x1
        rel_y = self.y2 - self.y1
        divisor = gcd(rel_x, rel_y)
        step_x = rel_x // divisor
        step_y = rel_y // divisor
        points = []
        for i in range(divisor + 1):
            # print(self.x1 + step_x*i, self.y1 + step_y*i)
            points.append((self.x1 + step_x * i, self.y1 + step_y * i))
        return points

In [5]:
# load line classes
lines = []
for input_coord in data:
    lines.append(Line(input_coord))

# find max_coord
max_coord_x = max([line.max_coord_x() for line in lines])
max_coord_y = max([line.max_coord_y() for line in lines])
# print(max_coord_x, max_coord_y)

# Q1
count_covered_points = np.zeros((max_coord_x+1,max_coord_y+1))
for line in lines:
    if not line.check_hor_or_ver():  # only consider horizontal and vertical lines
        continue
    for point in line.covered_points():
        count_covered_points[point] +=1
print(f"Answer to Q1: {(count_covered_points>1).sum()}")

# Q2
count_covered_points = np.zeros((max_coord_x+1,max_coord_y+1))
for line in lines:
#     if not line.check_hor_or_ver():  # only consider horizontal and vertical lines
#         continue
    for point in line.covered_points():
        count_covered_points[point] +=1
print(f"Answer to Q2: {(count_covered_points>1).sum()}")

Answer to Q1: 6564

Answer to Q2: 19172